In [1]:
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score
from functools import partial
from gnn_sAndp_weekly import startGNN
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from sklearn.model_selection import LeaveOneOut
import optuna

f:\miniconda3\envs\forGNN\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with_gold = False
with_oil = False
loss_function = 'CrossEntrophyLoss'
with_macd_signal = True
start_lr = 0.00018691291168759284
num_neighbors = 18
macdParamOptimize = False
gamma = 2
withAlpha = False
aggr = 'weight'
corr = 0.6
acc, loss = startGNN(start_lr, with_gold, with_oil, num_neighbors, loss_function, withMacdSignal=with_macd_signal, macdParamOptimize=macdParamOptimize, gamma=gamma, withAlpha=withAlpha, aggr=aggr, corr=corr)

In [2]:
username = "admin"
password = "admin"
server = "127.0.0.1"
database = "OptunaPortfolioGNN"

storage_url = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

In [ ]:
#study_name='param_optimize' goPredict(start_lr, False,False, num_neighbors, 'CrossEntrophyLoss', True, macdParamOptimize, 2, True)
#study_name='GNN1' goPredict(start_lr, False,False, num_neighbors, 'CrossEntrophyLoss', False, False, 2, True)
#study_name='GNN2' goPredict(start_lr, False,False, num_neighbors, 'FocalLoss', False, False, gamma, withAlpha)
#study_name='GNN3' goPredict(start_lr, False, True, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN4' goPredict(start_lr, True, False, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN5' goPredict(start_lr, True, True, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN6' goPredict(start_lr, True, True, num_neighbors, 'CrossEntrophyLoss', True, False, 2, False)
#study_name='GNN7' goPredict(start_lr, True, True, num_neighbors, 'CrossEntrophyLoss', True, True, 2, False)

In [ ]:
def goPredict(start_lr, with_gold,with_oil, num_neighbors, loss_function, with_macd_signal, macdParamOptimize, gamma, withAlpha, aggr='mean', corr=0.7, begin_days=1096):
    acc, loss = startGNN(start_lr, with_gold, with_oil, num_neighbors, loss_function, withMacdSignal=with_macd_signal, macdParamOptimize=macdParamOptimize, gamma=gamma, withAlpha=withAlpha, aggr=aggr, corr=corr, begin_days=begin_days)
    return loss
def objective(trial):
    # start_lr = trial.suggest_float('start_lr', 1e-5, 1e-3, log=True)
    # with_gold = trial.suggest_categorical('with_gold', [True, False])
    # with_oil = trial.suggest_categorical('with_oil', [True, False])
    num_neighbors = trial.suggest_int('num_neighbors', 18, 26)
    # loss_function = trial.suggest_categorical('loss_function', ['CrossEntrophyLoss', 'FocalLoss'])
    # with_macd_signal = trial.suggest_categorical('with_macd_signal', [True, False])
    # macdParamOptimize = trial.suggest_categorical('macdParamOptimize', [True, False]) #with_macd_signal要為true才有作用
    # gamma = trial.suggest_int('gamma', 1, 5) #loss_function = FocalLoss才有用
    # withAlpha = trial.suggest_categorical('withAlpha', [True, False]) #loss_function = FocalLoss才有用
    begin_days = trial.suggest_int('begin_days', 0, 1096)  
    loss = goPredict(0.00018691291168759284, True, False, num_neighbors, 'CrossEntrophyLoss', True, False, 2, False, aggr='weight', corr=0.6, begin_days=begin_days)
    return loss



In [ ]:
study = optuna.create_study(direction='minimize', study_name='GNN8', storage=storage_url, load_if_exists=True)
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best hyperparameters:", best_params)

In [ ]:
#study_name='GNN1' goPredict(start_lr, False,False, num_neighbors, 'CrossEntrophyLoss', False, False, 2, True)
#study_name='GNN2' goPredict(start_lr, False,False, num_neighbors, 'FocalLoss', False, False, gamma, withAlpha)
#study_name='GNN3' goPredict(start_lr, False, True, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN4' goPredict(start_lr, True, False, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN5' goPredict(start_lr, True, True, num_neighbors, 'CrossEntrophyLoss', False, False, 2, False)
#study_name='GNN6' goPredict(start_lr, False, False, num_neighbors, 'CrossEntrophyLoss', True, False, 2, False)
#study_name='GNN7' goPredict(start_lr, False, False, num_neighbors, 'CrossEntrophyLoss', True, True, 2, False)

# goPredict(0.00023308287052684873, False,False, 13, 'CrossEntrophyLoss', False, False, 2, True) #GNN1
# goPredict(0.00028935219407641655, False,False, 13, 'FocalLoss', False, False, 5, False) #GNN2
# goPredict(0.00043080401342612644, False, True, 15, 'CrossEntrophyLoss', False, False, 2, False) #GNN3
# goPredict(0.00023583717378475278, True, False, 13, 'CrossEntrophyLoss', False, False, 2, False) #GNN4
# goPredict(0.00024545748402171694, True, True, 13, 'CrossEntrophyLoss', False, False, 2, False) #GNN5
# goPredict(0.00018691291168759284, False, False, 18, 'CrossEntrophyLoss', True, False, 2, False) #GNN6
# goPredict(0.00018691291168759284, False, False, 24, 'CrossEntrophyLoss', True, False, 2, False, begin_days=653) #GNN6 new
# goPredict(0.00018691291168759284, False, False, 24, 'CrossEntrophyLoss', True, True, 2, False, begin_days=653) #GNN7 new
# goPredict(0.00018691291168759284, False, False, 24, 'CrossEntrophyLoss', True, False, 2, False, aggr='weight', corr=0.6, begin_days=653) #GNN8

In [6]:
load_study = optuna.load_study(study_name='GNN6', storage=storage_url)
completed_trials = [trial for trial in load_study.trials if trial.state == optuna.trial.TrialState.COMPLETE]

print(f"已經完成的trial數量: {len(completed_trials)}")

已經完成的trial數量: 50


In [ ]:
load_study.best_value

In [ ]:
load_study.best_params

In [8]:
optuna.visualization.plot_slice(load_study,params=["start_lr"], target_name='Loss')